In [ ]:
import os
import json

from PIL import Image

import torch
import torch.utils.data as data # для использования классов Dataset и Dataloader
import torchvision
import torchvision.transforms.v2 as tfs # для преобразования изображения в тензор

import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm # визуализация процесса обучения

from torchvision.datasets import ImageFolder # можно использовать вместо класса Dataset
# класс ImageFolder представляет изображения в RGB, то есть цветовых каналов будет 3
# структура папок с изображениями должна быть всегда одинаковой

In [9]:
class MNISTDigitNN(nn.Module):
    def __init__(self, input_dim, num_hidden, output_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, num_hidden)
        self.layer2 = nn.Linear(num_hidden, output_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = nn.functional.relu(x) # функция активации ReLU
        x = self.layer2(x)

        return x
    

model = MNISTDigitNN(28 * 28, 32, 10) # задаем количество нейронов на каждом слое

In [ ]:
# можно задать свое собственное преобразование

class RavelTransform(nn.Module):
    def forward(swlf, item):
        return item.ravel()

# пример применения
# transforms = tfs.Compose(RavelTransform()) 

In [18]:
# трансформация: изображение -> тензор -> градации серого (1 цветовой канал) -> 
# -> dtype=torch.float32 + нормировка (от 0 до 1) -> вытягиваем в один вектор
transforms = tfs.Compose([tfs.ToImage(), tfs.Grayscale(),
                           tfs.ToDtype(torch.float32, scale=True),
                            tfs.Lambda(lambda _img: _img.ravel())]) 
# при использовании полносвязных слоев, входное изображение вытягивают в один вектор при подаче на вход сети
d_train = ImageFolder("dataset/train", transform=transforms)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=True) # batch_size - 32 файла, shuffle - перемешиваем

it = iter(train_data)
x, y = next(it) # next(it) - один батч

print(x.size())

torch.Size([32, 784])


In [ ]:
optimizer = optim.Adam(params=model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()
epochs = 2

model.train()

best_loss = 1e10 # лучшее значение функции потерь, изначально установлено очень большое

for _e in range(epochs):
    loss_mean = 0 # среднее значение функции потерь, выводим в консоль в процессе обучения
    lm_count = 0

    train_tqdm = tqdm(train_data, leave=True)
    for x_train, y_train in train_tqdm:
        predict = model(x_train)
        loss = loss_function(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1/lm_count * loss.item() + (1 - 1/lm_count) * loss_mean # рекрентная формула
        train_tqdm.set_description(f"Epoch [{_e+1}/{epochs}], loss_mean={loss_mean:.3f}")

    if best_loss > loss_mean * 1.1:
        # выполняем сохранение модели, если среднее значение функции потерь уменьшилось на 10%
        best_loss = loss_mean
        st = model.state_dict()
        torch.save(st, f'model_MNIST_{_e}.tar')

Epoch [2/2], loss_mean=0.186: 100%|██████████| 1875/1875 [00:30<00:00, 61.16it/s]


In [ ]:
# загрузка модели

st2 = torch.load('model_MNIST.tar', weights_only=True, map_location="cpu") 
# weights_only=True - выполняется загрузка примитивных типов данных
# map_location="cpu" загружаем на CPU, можно выбрать map_location="cuda"

model.load_state_dict(st2) # передаем веса и смещения в модель

<All keys matched successfully>

In [24]:
d_test = ImageFolder("dataset/test", transform=transforms) # тестовая выборка
test_data = data.DataLoader(d_test, batch_size=500, shuffle=False) # для тестирования не имеет смысла перемешивать выборку

Q = 0

# тестирование обученной НС
model.eval()

for x_test, y_test in test_data:
    with torch.no_grad():
        p = model(x_test)
        p = torch.argmax(p, dim=1) 
        Q += torch.sum(p == y_test).item()

Q /= len(d_test) # доля правильных ответов
print(Q)

0.9445
